## Part 3: Predictions


In [2]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import pickle
import utils as u
from time import time
from sklearn.externals import joblib
# sklearn
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [51]:
mailout_test = pd.read_csv('Udacity_MAILOUT_052018_TEST.csv', sep=';')
mailout_test.drop(columns='Unnamed: 0', inplace=True)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [52]:
import importlib
importlib.reload(u)

<module 'utils' from '/Volumes/Mac HD 1TB/Identify_CustomerSegments/Arvato Project/utils.py'>

In [4]:
# Also the attribute file will come in handy for handling missing or unknown values
dictionary = pd.read_excel('DIAS Attributes - Values 2017_revised.xlsx', sheet_name='Tabelle1')
dictionary.drop(columns='Unnamed: 0', inplace=True)

### Preprocessing test data

In [5]:
column_dist = mailout_test.isnull().sum()
column_dist.sort_values(ascending=False, inplace=True)
outlier_columns = column_dist.index[:8]
print(column_dist[:30]/mailout_test.shape[0])

ALTER_KIND4       0.999089
ALTER_KIND3       0.995307
ALTER_KIND2       0.982210
ALTER_KIND1       0.953004
KK_KUNDENTYP      0.584479
EXTSEL992         0.369085
HH_DELTA_FLAG     0.224570
W_KEIT_KIND_HH    0.224570
KBA05_MAXVORB     0.199309
KBA05_MOTOR       0.199309
KBA05_MOD4        0.199309
KBA05_MOD3        0.199309
KBA05_MOD2        0.199309
KBA05_MOD1        0.199309
KBA05_MAXSEG      0.199309
KBA05_KRSZUL      0.199309
KBA05_MOTRAD      0.199309
KBA05_MAXBJ       0.199309
KBA05_MAXAH       0.199309
KBA05_KW3         0.199309
KBA05_KW2         0.199309
MOBI_REGIO        0.199309
KBA05_KW1         0.199309
KBA05_MAXHERST    0.199309
KBA05_ZUL3        0.199309
KBA05_SEG1        0.199309
KBA05_KRSOBER     0.199309
KBA05_ZUL4        0.199309
KBA05_ZUL2        0.199309
KBA05_ZUL1        0.199309
dtype: float64


In [6]:
cat_col = u.obtain_categorical_columns(mailout_test)
cat_col

{'multi': ['LP_FAMILIE_FEIN',
  'LP_FAMILIE_GROB',
  'LP_STATUS_FEIN',
  'LP_STATUS_GROB',
  'NATIONALITAET_KZ',
  'SHOPPER_TYP',
  'TITEL_KZ',
  'VERS_TYP',
  'CJT_GESAMTTYP',
  'CAMEO_DEUG_2015',
  'FINANZTYP',
  'GEBAEUDETYP',
  'GFK_URLAUBERTYP',
  'ZABEOTYP'],
 'binary': ['OST_WEST_KZ', 'ANREDE_KZ', 'GREEN_AVANTGARDE']}

In [7]:
unknown_dict = u.create_unknown_dictionary(dictionary)

Shape of the attribute file: (2258, 4)
Missing values in the array: 11
After filling NaNs with False: 
False    2025
True      233
Name: Meaning, dtype: int64


In [53]:
#Cleaning 
mailout_test, LNR = u.clean_mailout(mailout_test.copy(), unknown_dict, 
                                                    outlier_columns, cat_col, train=False)

In [55]:
# Creating X_train
X_test = mailout_test.drop(columns=['D19_LETZTER_KAUF_BRANCHE'])
X_test.head()

,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,...,ZABEOTYP_2,ZABEOTYP_3,ZABEOTYP_4,ZABEOTYP_5,ZABEOTYP_6,PRAEGENDE_JUGENDJAHRE_MOVEMENT,PRAEGENDE_JUGENDJAHRE_DECADE,CAMEO_INTL_2015_WEALTH,CAMEO_INTL_2015_LIFESTAGE,EINGEFUEGT_AM_YEAR
0,2.0,1.0,7.0,6.0,2.0,0.0,0.0,2.0,2.0,0.0,...,0,1,0,0,0,1.0,40.0,1.0,3.0,1993.0
1,NaN,1.0,NaN,0.0,20.0,0.0,0.0,1.0,21.0,0.0,...,0,1,0,0,0,0.0,60.0,3.0,1.0,1992.0
2,2.0,9.0,16.0,11.0,2.0,0.0,0.0,4.0,2.0,0.0,...,0,1,0,0,0,1.0,60.0,4.0,1.0,1992.0
3,NaN,7.0,NaN,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0,1,0,0,0,0.0,60.0,1.0,3.0,1992.0
4,1.0,1.0,21.0,13.0,1.0,0.0,0.0,4.0,1.0,0.0,...,0,0,1,0,0,0.0,70.0,3.0,1.0,1992.0


### Loading models

In [65]:
# Loading models
ada = joblib.load('ada_model.pkl')
rf = joblib.load('rf_model.pkl')
sgd = joblib.load('sgd_model.pkl')
gbc = joblib.load('gbc_model.pkl')

### Predictions

In [68]:
# AdaBoost
ada_pred = ada.predict_proba(X_test)
# Random Forest
rf_pred = rf.predict_proba(X_test)
# SGD
sgd_pred = sgd.predict_proba(X_test)
# Gradient Boosting
gbc_pred = gbc.predict_proba(X_test)

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [72]:
submission_ada = pd.DataFrame(ada_pred[:,1], index=LNR, columns=['RESPONSE'])
submission_ada.head(5)

,RESPONSE
LNR,
1754,0.031359
1770,0.045165
1465,0.001424
1470,0.001258
1478,0.002888


In [73]:
submission_gbc = pd.DataFrame(gbc_pred[:,1], index=LNR, columns=['RESPONSE'])
submission_gbc.head(5)

,RESPONSE
LNR,
1754,0.021005
1770,0.016986
1465,0.001711
1470,0.001342
1478,0.001321


In [77]:
submission_rf = pd.DataFrame(rf_pred[:,1], index=LNR, columns=['RESPONSE'])
submission_rf.head(5)

,RESPONSE
LNR,
1754,0.020833
1770,0.003333
1465,0.022000
1470,0.002500
1478,0.037095


In [64]:
submission_ada.to_csv('ada_submission.csv')
submission_gbc.to_csv('gbc_submission.csv')

After making the submission, the model that performed best was Gradient Boosting Classifier, with a score of 0.789 compared to the 0.808, which is the lead score. There might be some ways to improve the score of the model, like adding the ``customers`` data to the mailout in order to have a better chance to have balance classes. 

#### References

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold
https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html#sphx-glr-auto-examples-compose-plot-column-transformer-mixed-types-py
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier
https://datascience.stackexchange.com/questions/32818/train-test-split-of-unbalanced-dataset-classification
https://www.migarage.ai/intelligence/imbalanced-data/